### Estimación de peso y dimensiones de los envíos de Mercado Libre

Este notebook es una versión mínima y necesaria para poder levantar el dataset [meli_dataset_20190426.csv](https://drive.google.com/open?id=1tNUKD1lf1z8C7LPpCruiDl6SKBAHb79v) y comprender los features que éste contiene.

#### Sobre el dataset
El dataset corresponde a un muestreo de envíos durante tres meses de 2018 en MLB (Mercado Libre Brasil), filtrados según métodos de envíos y por carriers (empresas de correos a cargo del envío propiamente dicho). A su vez, fue muestreado de forma no uniforme para romper de forma manual la distribución de los datos.

In [35]:
import pandas as pd
import random

In [36]:
random.seed(0)

In [37]:
DATASET = 'meli_dataset_20190426.csv'

In [38]:
df = pd.read_csv(DATASET, low_memory=False)

In [39]:
df.columns

Index(['ITEM_ID', 'SHP_WEIGHT', 'SHP_LENGTH', 'SHP_WIDTH', 'SHP_HEIGHT',
       'ATTRIBUTES', 'CATALOG_PRODUCT_ID', 'CONDITION', 'DOMAIN_ID', 'PRICE',
       'SELLER_ID', 'STATUS', 'TITLE'],
      dtype='object')

#### ¿Qué significa cada columna?

* ITEM_ID: id unívoco de cada item publicado. (Ofuscado)
* SHP_WEIGHT: peso del paquete informado por el correo.
* SHP_LENGTH: largo del paquete informado por el correo.
* SHP_WIDTH: ancho del paquete informado por el correo.
* SHP_HEIGHT: altura del paquete informado por el correo.
* ATTRIBUTES: atributos como marca y modelo, entre otros, en formato json-lines
* CATALOG_PRODUCT_ID: id del catálogo (ofuscado).
* CONDITION: condición de venta (nuevo o usado).
* DOMAIN_ID: id de la categoría a la que pertenece la publicación.
* PRICE: precio en reales.
* SELLER_ID: id del vendedor (ofuscado).
* STATUS: estado de la publicación (activa, cerrada, pausada, etc.)
* TITLE: título de la publicación.

In [40]:
df.sample(10)

,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,ATTRIBUTES,CATALOG_PRODUCT_ID,CONDITION,DOMAIN_ID,PRICE,SELLER_ID,STATUS,TITLE
221378,C0R80IG7C9,1665.0,26.0,16.0,16.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-SUPPLEMENTS,119.95,G5IOIO0WVP,active,Kit: Whey Protein Concentrado 1kg + Vitamina C...
238027,FZPATZH1GK,150.0,30.0,15.0,5.0,"[{'id': 'ALPHANUMERIC_MODEL', 'name': 'Modelo ...",H53U1H7Q5G,new,MLB-WRISTWATCHES,158.00,XI83K3713D,active,Relogio Masculino Pulseira Couro Militar Sport
85828,TKA4ZILOY1,1280.0,34.0,25.0,10.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-TOYS_AND_GAMES,55.99,SYDSAO9TSX,active,Pneu Mini Moto Cross Ninja 49cc 12 1/2 2.75 Ar...
116866,PZWQTQ7XQY,220.0,16.0,11.0,10.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-FUEL_INJECTORS,240.00,ABLWCR3DDY,active,4 Bicos Injetores Iwp 026 Scénic/clio 1.6 Mag...
346456,IBS6RR7WPI,530.0,51.0,43.0,15.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-BABY_TEETHERS,64.90,T8ONWQRA69,active,Tapete Infantil Portátil Atividades Para Bebê ...
37476,PNVUEKIMHV,1120.0,33.0,28.0,15.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-CAR_TAIL_LIGHTS,197.90,HF7WXL6PWQ,active,Lanterna Traseira Ecosport 2003 2004 2005 2006...
170567,BKX79TZ2AD,NaN,NaN,NaN,NaN,"[{'id': 'BATTERY_CAPACITY', 'name': 'Capacidad...",F6QG5TZZSJ,new,MLB-CELLPHONE_BATTERIES,75.59,WFVVWSX9TF,closed,Bateria C11p1501 Asus Zenfone Zd551kl Ze550kl ...
152685,Y5P2NQ140C,NaN,NaN,NaN,NaN,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-VEHICLE_LED_BULBS,99.90,M6CA9GND49,paused,Caixa De Som Bluetooth Charge 3 Jbl
139568,T6TRIMFI57,330.0,25.0,25.0,5.0,"[{'id': 'BOTTOM_CUT', 'name': 'Corte da parte ...",H53U1H7Q5G,new,MLB-JUMPSUITS_AND_OVERALLS,48.32,JWGUVUUC1Z,active,Macaquinho Macacão Jogger Cintura Marcada Bojo...
158721,D79C0KCY9R,300.0,0.0,0.0,0.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-PANTS,39.50,XSWPBAH7J9,active,Calcinha Levanta Bumbum Material Spandex


In [44]:
df = df.dropna(subset=['SHP_WEIGHT'])
df = df.dropna(subset=['SHP_LENGTH'])
df = df.dropna(subset=['SHP_WIDTH'])
df = df.dropna(subset=['SHP_HEIGHT'])
df = df.dropna(subset = ['SHP_WEIGHT', 'SHP_LENGTH', 'SHP_WIDTH', 'SHP_HEIGHT'])
df.sample(300)

,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,ATTRIBUTES,CATALOG_PRODUCT_ID,CONDITION,DOMAIN_ID,PRICE,SELLER_ID,STATUS,TITLE
377994,ACO05SH1P8,566.0,21.0,21.0,9.0,"[{'id': 'BODY_CREAM_FUNCTION', 'name': 'Função...",H53U1H7Q5G,new,MLB-BODY_CREAMS,127.90,PL2VGHW7JW,closed,Gel E Sérum Pimenta Negra Termo Ativo D'agua N...
270996,SL1YC6LI3Z,850.0,20.0,16.0,12.0,"[{'id': 'GTIN', 'name': 'Código universal do p...",H53U1H7Q5G,new,MLB-TV_REMOTE_CONTROLS,23.99,PXCG7YPY9M,closed,Extensor Repetidor De Controle Remoto Ir Tv Dv...
376216,GQAHD51QZA,340.0,33.0,14.0,11.0,NaN,H53U1H7Q5G,NaN,NaN,NaN,O3CPLCSOB5,inactive,Cera Creme Refil Plástico Vazio Roll On Santa ...
99223,VB9C4I0R3S,7460.0,49.0,41.0,40.0,NaN,H53U1H7Q5G,NaN,NaN,NaN,WXKNBEZ36F,under_review,Mini Sofa Infantil (poltrona. Sofazinho)
164043,MVLRHEG0X9,930.0,16.0,11.0,4.0,"[{'id': 'ALPHANUMERIC_MODEL', 'name': 'Modelo ...",H53U1H7Q5G,new,MLB-HAIR_STRAIGHTENERS,64.99,HWEUQ8T8XG,active,Chapinha Prancha Profissional Nano Titanium A...
185084,RCHXJL6OZ7,244.0,25.0,25.0,5.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-T_SHIRTS,39.90,VO35WCEJFE,active,Camiseta Helloween Keeper Of The Seven The Leg...
394982,B8IUAHBHLY,780.0,41.0,30.6,26.6,"[{'id': 'AMBIENTS', 'name': 'Ambientes', 'valu...",H53U1H7Q5G,new,MLB-CEILING_LIGHTS,48.00,BCCJ5AF4WW,active,1 Lustre Pendente Luminária Aramado Diamante P...
245233,I61V8QJB0T,258.0,27.0,17.0,12.0,"[{'id': 'ITEM_CONDITION', 'name': 'Condição do...",H53U1H7Q5G,new,MLB-VEHICLE_ACCESSORIES,52.80,TN4KINMAAR,active,Entrada De Ar Decorativa Cromada Colméia 2 Par...
295224,VAEKBEMOT3,7000.0,20.0,15.0,15.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-SCHOOL_AND_OFFICE_PAPERS,113.99,ABERMZYL92,active,Papel Fotográfico 180g Glossy A4 À 500 Folhas...
154759,LBN83NUJ52,590.0,25.0,11.0,5.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",L9PYMYWPR8,new,MLB-SMARTWATCHES,1658.00,QN79KFBD4W,active,Apple Watch Series 3 42mm Gps Prova D´água Pr...
